# Dependencies

In [ ]:
!python -c "import torch; print(torch.__version__)"

In [ ]:
%%capture
# Download the corresponding PyTorch Geometric module
"""
Assign to TORCH with what you get from the cell above. E.g., export TORCH=1.12.1+cu113
"""
%env TORCH=2.1.0+cu121
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric

In [ ]:
import os
import numpy as np
import random
from tqdm import tqdm
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
import torch.optim as optim

from torch_geometric.loader import DataLoader, NeighborLoader
from torch_geometric.data.data import Data
import torch_geometric.utils as U
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.nn import MessagePassing, GraphConv, GCNConv, GINConv, global_mean_pool
from torch_geometric.utils.convert import from_scipy_sparse_matrix, from_networkx, to_networkx

import matplotlib.pyplot as plt
import networkx as nx

from scipy.stats.stats import pearsonr

# Data utilities

## Load or construct data

In [ ]:
def get_synth_data(max_size=9):
    """
    Returns lists of adjacency matrices for a dataset consisting of various
    synthetic graphs.
    """
    A = []

    for matrix_size in range(3, max_size+1):
      #cycle graphs
      cycle_graph = torch.zeros((matrix_size, matrix_size))

      for i in range(matrix_size - 1):
          cycle_graph[i, i+1] = 1
      A.append(cycle_graph + cycle_graph.T)

      #grid graphs

      #complete graphs
      A.append(torch.ones((matrix_size, matrix_size)))

      #star graphs
      star_graph = torch.zeros((matrix_size, matrix_size), dtype=int)
      star_graph[0, 1:] = 1
      star_graph[1:, 0] = 1
      A.append(star_graph)

      #E-R graphs
      for _ in range(8):
        edge_probability = 0.5
        e_r_graph = torch.zeros((matrix_size, matrix_size), dtype=int)

        for i in range(matrix_size):
            for j in range(i + 1, matrix_size):
                if np.random.rand() < edge_probability:
                    e_r_graph[i, j] = 1
                    e_r_graph[j, i] = 1
        A.append(e_r_graph)

        #B-A graphs
        m = 2
        ba_graph = nx.barabasi_albert_graph(matrix_size, m)
        ba_graph_adj = torch.from_numpy(nx.to_numpy_array(ba_graph))
        A.append(ba_graph_adj)

        #W-S graphs
        k = 2
        p = 0.2
        ws_graph = nx.watts_strogatz_graph(matrix_size, k, p)
        ws_graph_adj = torch.from_numpy(nx.to_numpy_array(ws_graph))
        A.append(ws_graph_adj)
    return A

In [ ]:
def get_dataset(dataset_name='MUTAG', node_features_type='original'):
    """
    Returns lists of adjacency matrices for a dataset in TUDataset.
    """
    raw_dataset = TUDataset(root='data/TUDataset', name=dataset_name)
    A = []
    X = []
    Y = []
    for graph in raw_dataset:
        adj_matrix = U.to_dense_adj(graph.edge_index).squeeze(0)
        A.append(adj_matrix)
        X.append(graph.x)
        Y.append(graph.y)

    if node_features_type == 'original':
      pass
    elif node_features_type == 'constant':
      X = [torch.ones(a.shape[0], 1) for a in A]
    elif node_features_type == 'random':
      X = [torch.rand(a.shape[0], 1) for a in A]
    return A

## Preprocess data

In [ ]:
def preprocess_graphs_ged(adj_matrix_list):
    n = len(adj_matrix_list)

    num_samples = n // 2
    for i in range(num_samples):
      sample = {}
      start = 2 * i
      end = min(2 * i + 1, n)

      sample['adj1'] = adj_matrix_list[start]
      sample['adj2'] = adj_matrix_list[end]

      graph1 = nx.from_numpy_array(sample['adj1'].numpy())
      graph2 = nx.from_numpy_array(sample['adj2'].numpy())

      norm_ged = 2 * next(nx.optimize_graph_edit_distance(graph1, graph2)) / (sample['adj1'].shape[0] + sample['adj2'].shape[0])
      #norm_ged = 2 * nx.graph_edit_distance(graph1, graph2) / (sample['adj1'].shape[0] + sample['adj2'].shape[0])
      sample['target'] = torch.from_numpy(np.array(np.exp(-norm_ged))).float()

      degrees1 = torch.tensor([graph1.degree(node) for node in graph1.nodes])
      degrees2 = torch.tensor([graph2.degree(node) for node in graph2.nodes])
      tiled_degrees1 = torch.tile(degrees1, (5, 1)).T
      tiled_degrees2 = torch.tile(degrees2, (5, 1)).T

      traingles1 = torch.tensor([float(trig) for trig in nx.triangles(graph1).values()])
      traingles2 = torch.tensor([float(trig) for trig in nx.triangles(graph2).values()])
      tiled_traingles1 = torch.tile(traingles1, (5, 1)).T
      tiled_traingles2 = torch.tile(traingles2, (5, 1)).T

      sample['feats1'] = torch.hstack((tiled_degrees1, tiled_traingles1))
      sample['feats2'] = torch.hstack((tiled_degrees2, tiled_traingles2))


      #sample['feats1'] = torch.ones((sample['adj1'].shape[0], 10))
      #sample['feats2'] = torch.ones((sample['adj2'].shape[0], 10))
      yield sample

def preprocess_graphs_feats(adj_matrix_list):
    n = len(adj_matrix_list)

    num_samples = n // 2
    for i in range(num_samples):
      sample = {}
      start = 2 * i
      end = min(2 * i + 1, n)

      sample['adj1'] = adj_matrix_list[start]
      sample['adj2'] = adj_matrix_list[end]

      graph1 = nx.from_numpy_array(sample['adj1'].numpy())
      graph2 = nx.from_numpy_array(sample['adj2'].numpy())

      trig_graph1 = 1 if max(nx.triangles(graph1).values()) >= 1 else 0
      trig_graph2 = 1 if max(nx.triangles(graph2).values()) >= 1 else 0

      avg_clust_coeff_graph1 = 1 if nx.average_clustering(graph1) >= 0.5 else 0
      avg_clust_coeff_graph2 = 1 if nx.average_clustering(graph2) >= 0.5 else 0

      girth_graph1 = 1 if nx.girth(graph1) <= 3 else 0
      girth_graph2 = 1 if nx.girth(graph2) <= 3 else 0

      diameter1 = 1 if max([max(j.values()) for (i,j) in nx.shortest_path_length(graph1)]) <= 4 else 0
      diameter2 = 1 if max([max(j.values()) for (i,j) in nx.shortest_path_length(graph2)]) <= 4 else 0

      even_nodes1 = 1 if sample['adj1'].shape[0] % 2 else 0
      even_nodes2 = 1 if sample['adj2'].shape[0] % 2 else 0

      eps = 1e-8
      feats1 = np.array([1., trig_graph1, diameter1, avg_clust_coeff_graph1, girth_graph1, diameter1, even_nodes1])
      feats2 = np.array([1., trig_graph2, diameter2, avg_clust_coeff_graph2, girth_graph2, diameter2, even_nodes2])
      # sample['target'] = 1. - torch.from_numpy(np.array(np.inner(feats1, feats2) / (np.linalg.norm(feats1) * np.linalg.norm(feats2)))).float()
      sample['target'] = 1. - nn.CosineSimilarity(dim=0, eps=eps)(torch.from_numpy(feats1).float(), torch.from_numpy(feats2).float())

      degrees1 = torch.tensor([graph1.degree(node) for node in graph1.nodes])
      degrees2 = torch.tensor([graph2.degree(node) for node in graph2.nodes])
      tiled_degrees1 = torch.tile(degrees1, (5, 1)).T
      tiled_degrees2 = torch.tile(degrees2, (5, 1)).T

      traingles1 = torch.tensor([float(trig) for trig in nx.triangles(graph1).values()])
      traingles2 = torch.tensor([float(trig) for trig in nx.triangles(graph2).values()])
      tiled_traingles1 = torch.tile(traingles1, (5, 1)).T
      tiled_traingles2 = torch.tile(traingles2, (5, 1)).T

      sample['feats1'] = torch.hstack((tiled_degrees1, tiled_traingles1))
      sample['feats2'] = torch.hstack((tiled_degrees2, tiled_traingles2))

      #sample['feats1'] = torch.ones((sample['adj1'].shape[0], 10))
      #sample['feats2'] = torch.ones((sample['adj2'].shape[0], 10))
      yield sample


# GNN Classes

## Baselines

In [ ]:
class ConstantBaseline(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, dropout=0.5,
                 mode='inner'):
        super(ConstantBaseline, self).__init__()

        self.const = torch.nn.Parameter(torch.rand(1))

    def forward(self, data) -> torch.Tensor:
        return self.const

In [ ]:
class RandomBaseline(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, dropout=0.5,
                 mode='inner'):
        super(RandomBaseline, self).__init__()
        self.const = torch.nn.Parameter(torch.rand(1))

    def forward(self, data) -> torch.Tensor:
        return torch.rand(1) + self.const - self.const

## GNNs

In [ ]:
class TensorBoard(nn.Module):
  def __init__(self,output_dim):
        super(TensorBoard, self).__init__()
        self.output_dim = output_dim
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.setup_weights()
        self.init_parameters()

  def setup_weights(self):
        """
        Defining weights.
        """
        self.weight_matrix = torch.nn.Parameter(torch.Tensor(self.output_dim,
                                                             self.output_dim))

        self.weight_matrix_block = torch.nn.Parameter(torch.Tensor(1,
                                                                   2 * self.output_dim))
        self.bias = torch.nn.Parameter(torch.Tensor(1, 1))

  def init_parameters(self):
        torch.nn.init.normal_(self.weight_matrix)
        torch.nn.init.normal_(self.weight_matrix_block)
        torch.nn.init.normal_(self.bias)

  def forward(self, embedding_1, embedding_2):
        scoring = torch.matmul(torch.t(embedding_1.view(self.output_dim, 1)), self.weight_matrix)
        scoring = torch.matmul(scoring, embedding_2.view(self.output_dim, 1))
        combined_representation = torch.cat((embedding_1, embedding_2)).view(2 * self.output_dim, 1)
        block_scoring = torch.matmul(self.weight_matrix_block, combined_representation)
        scores = torch.nn.functional.relu(scoring + block_scoring + self.bias)
        return scores

In [ ]:
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, dropout=0.5,
                 mode='inner'):
        super(GNN, self).__init__()
        """
        Args:
            input_dim: input feaure dimension
            hidden_dim: hidden feature dimension
            output_dim: output dimensions
            n_layers: number of layers
            dropout: dropout_ratio
        """
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.dropout = dropout
        self.mode = mode
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.conv_layers = nn.ModuleList([GraphConv(input_dim, hidden_dim)])
        for i in range(1,n_layers):
            self.conv_layers.append(GraphConv(hidden_dim, hidden_dim))

        if mode not in ['inner', 'tensor_board']:
          raise NotImplementedError('Please use a valid mode type')

        if mode == 'tensor_board':
          self.tensor_board = TensorBoard(self.output_dim)
        self.linear = nn.Linear(hidden_dim, output_dim)

    def conv_pass(self, x, adj):
        edge_index = U.dense_to_sparse(adj)[0].to(self.device)

        for i in range(self.n_layers):
            x = self.conv_layers[i](x, edge_index)

            if i < self.n_layers - 1:
                x = F.relu(x) #Remove ReLU for the last layer

            x = F.dropout(x, p=self.dropout, training = self.training)

        x = self.linear(x)
        return x

    def forward(self, data) -> torch.Tensor:
        adj1 = data["adj1"]
        adj2 = data["adj2"]
        feats1 = data["feats1"]
        feats2 = data["feats2"]

        latent_feats1 = self.conv_pass(feats1, adj1)
        latent_feats2 = self.conv_pass(feats2, adj2)

        pooled_feats1 = torch.mean(latent_feats1, dim=0)
        pooled_feats2 = torch.mean(latent_feats2, dim=0)

        if self.mode == 'inner':
          inner = 1. - torch.inner(pooled_feats1, pooled_feats2) / (torch.norm(pooled_feats1) * torch.norm(pooled_feats2))
          return inner
        else:
          scoring = self.tensor_board(pooled_feats1, pooled_feats2)
          return scoring


In [ ]:
import math

def init_params(tensor_size: tuple, mean: float = 0.0, std: float = 1.0
                ) -> nn.Parameter:
    weights = nn.Parameter(torch.FloatTensor(tensor_size[0], tensor_size[1]))
    stdv = 1. / math.sqrt(weights.size(1))
    weights.data.uniform_(-stdv, stdv)
    return weights

In [ ]:
class GNNLayerGlobalReadout(nn.Module):
    def __init__(self, input_dim, output_dim):
      super(GNNLayerGlobalReadout, self).__init__()

      # read in parameters
      self.input_dim, self.output_dim = input_dim, output_dim

      # initialise weights
      self.weights_self = init_params(tensor_size=(self.input_dim,
                                                   self.output_dim),
                                      mean=0.0, std=1.0)
      self.weights_neigh = init_params(tensor_size=(self.input_dim,
                                                   self.output_dim),
                                      mean=0.0, std=1.0)
      self.weights_global = init_params(tensor_size=(self.input_dim,
                                                   self.output_dim),
                                      mean=0.0, std=1.0)


    def forward(self, node_feats, adj_matrix):
      self_term = torch.matmul(node_feats, self.weights_self)
      neigh_term = torch.matmul(adj_matrix.float(), torch.matmul(node_feats,
                                                 self.weights_neigh))
      global_term = torch.matmul(torch.ones(adj_matrix.shape), torch.matmul(node_feats,
                                                 self.weights_global))
      return torch.add(torch.add(self_term, neigh_term), global_term)

In [ ]:
class GNNGlobalReadout(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, dropout=0.5,
                 mode='inner'):
        super(GNNGlobalReadout, self).__init__()
        """
        Args:
            input_dim: input feaure dimension
            hidden_dim: hidden feature dimension
            output_dim: output dimensions
            n_layers: number of layers
            dropout: dropout_ratio
        """
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.dropout = dropout
        self.mode = mode
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.conv_layers = nn.ModuleList([GNNLayerGlobalReadout(input_dim, hidden_dim)])
        for i in range(1,n_layers):
            self.conv_layers.append(GNNLayerGlobalReadout(hidden_dim, hidden_dim))

        if mode not in ['inner', 'tensor_board']:
          raise NotImplementedError('Please use a valid mode type')

        if mode == 'tensor_board':
          self.tensor_board = TensorBoard(self.output_dim)
        self.linear = nn.Linear(hidden_dim, output_dim)

    def conv_pass(self, x, adj):
        edge_index = U.dense_to_sparse(adj)[0].to(self.device)

        for i in range(self.n_layers):
            x = self.conv_layers[i](x, adj)

            if i < self.n_layers - 1:
                x = F.relu(x) #Remove ReLU for the last layer

            x = F.dropout(x, p=self.dropout, training = self.training)

        x = self.linear(x)
        return x

    def forward(self, data) -> torch.Tensor:
        adj1 = data["adj1"]
        adj2 = data["adj2"]
        feats1 = data["feats1"]
        feats2 = data["feats2"]

        latent_feats1 = self.conv_pass(feats1, adj1)
        latent_feats2 = self.conv_pass(feats2, adj2)

        pooled_feats1 = torch.mean(latent_feats1, dim=0)
        pooled_feats2 = torch.mean(latent_feats2, dim=0)

        if self.mode == 'inner':
          inner = 1. - torch.inner(pooled_feats1, pooled_feats2) / (torch.norm(pooled_feats1) * torch.norm(pooled_feats2))
          return inner
        else:
          scoring = self.tensor_board(pooled_feats1, pooled_feats2)
          return scoring


In [ ]:
class GIN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, dropout=0.5,
                 mode='inner'):
        super(GIN, self).__init__()
        """
        Args:
            input_dim: input feaure dimension
            hidden_dim: hidden feature dimension
            output_dim: output dimensions
            n_layers: number of layers
            dropout: dropout_ratio
        """
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.output_dim = output_dim
        self.dropout = dropout
        self.mode = mode
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.conv_layers = nn.ModuleList([GINConv(
            Sequential(Linear(self.input_dim, self.hidden_dim),
                       BatchNorm1d(self.hidden_dim), ReLU(),
                       Linear(self.hidden_dim, self.hidden_dim), ReLU()))])
        for i in range(1, self.n_layers):
          self.conv_layers.append(GINConv(
                                  Sequential(Linear(self.hidden_dim, self.hidden_dim), BatchNorm1d(self.hidden_dim), ReLU(),
                                            Linear(self.hidden_dim, self.hidden_dim), ReLU())))
        if mode not in ['inner', 'tensor_board']:
          raise NotImplementedError('Please use a valid mode type')

        if mode == 'tensor_board':
          self.tensor_board = TensorBoard(self.output_dim)
        self.linear = nn.Linear(hidden_dim, output_dim)

    def conv_pass(self, x, adj):
        edge_index = U.dense_to_sparse(adj)[0].to(self.device)

        for i in range(self.n_layers):
            x = self.conv_layers[i](x, edge_index)

        x = self.linear(x)
        return x

    def forward(self, data) -> torch.Tensor:
        adj1 = data["adj1"]
        adj2 = data["adj2"]
        feats1 = data["feats1"]
        feats2 = data["feats2"]

        latent_feats1 = self.conv_pass(feats1, adj1)
        latent_feats2 = self.conv_pass(feats2, adj2)

        pooled_feats1 = torch.mean(latent_feats1, dim=0)
        pooled_feats2 = torch.mean(latent_feats2, dim=0)

        if self.mode == 'inner':
          inner = 1. - torch.inner(pooled_feats1, pooled_feats2) / (torch.norm(pooled_feats1) * torch.norm(pooled_feats2))
          return inner
        else:
          scoring = self.tensor_board(pooled_feats1, pooled_feats2)
          return scoring

# Training and evaluation setup

In [ ]:
def train(data_iterator, model, optimizer, criterion, device):
    model.train()

    losses = []
    targets = []
    predictions = []
    for data in data_iterator:
        optimizer.zero_grad()

        pred = model(data).to(device)

        loss = criterion(pred, data['target'])
        losses.append(loss.item())
        predictions.append(pred.detach().numpy())
        targets.append(data['target'].detach().numpy())

        loss.backward()
        optimizer.step()
    return np.mean(losses), np.corrcoef(np.array(predictions).flatten(), np.array(targets))[0, 1]

def eval(data_iterator, model, device):
    model.eval()

    losses = []
    targets = []
    predictions = []

    for data in data_iterator:

        pred = model(data).to(device)

        loss = torch.mean(torch.square(pred - data['target']))
        losses.append(loss.item())

        predictions.append(pred.detach().numpy())
        targets.append(data['target'].detach().numpy())

    return np.mean(losses), np.corrcoef(np.array(predictions).flatten(), np.array(targets))[0, 1]

In [ ]:
def train_model(A_train, A_test, params, verbose):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    model = params['model'](params['input_dim'],
                            params['hidden_dim'],
                            params['output_dim'],
                            params['n_layers'],
                            params['dropout'],
                            params['mode']).to(device)

    optimizer = optim.Adam(model.parameters(),
                           lr=params['lr'],
                           weight_decay=params['weight_decay'])

    criterion = nn.MSELoss()

    early_stopper = EarlyStopper(params['max_patience'])

    train_losses = []
    test_losses = []

    train_correlations = []
    test_correlations = []

    for epoch in tqdm(range(1, params['epochs']+1)):
        if params['similarity_measure'] == 'ged':
          train_data = preprocess_graphs_ged(A_train)
          test_data = preprocess_graphs_ged(A_test)
        elif params['similarity_measure'] == 'feats':
          train_data = preprocess_graphs_feats(A_train)
          test_data = preprocess_graphs_feats(A_test)
        else:
          raise NotImplementedError('Please use a valid Similarity Measure')
        train_loss, train_correlation = train(train_data,
                          model,
                          optimizer,
                          criterion,
                          device)
        test_loss, test_correlation = eval(test_data,
                        model,
                        device)

        epoch_len = len(str(params['epochs']))

        print_msg = (f'[{epoch:>{epoch_len}}/{params["epochs"]:>{epoch_len}}] ' +
                        f'loss: {train_loss:.5f} ' +
                        f'test loss: {test_loss:.5f} ' +
                        f'train corrlation: {train_correlation}'
                        f'test corrlation: {test_correlation}'
                        )
        if verbose:
            print(print_msg)

        train_losses.append(train_loss)
        test_losses.append(test_loss)
        train_correlations.append(train_correlation)
        test_correlations.append(test_correlation)

    print('Train loss: {}'.format(train_losses[-1]),
          'Test loss: {}'.format(test_losses[-1]),
          'Train correlation: {}'.format(train_correlations[-1]),
          'Test correlation: {}'.format(test_correlations[-1]))
    return train_losses[-1], test_losses[-1-1], train_correlations[-1], test_correlations[-1]

In [ ]:
def cross_val(A, params, verbose=False):
    """
    10-fold cross-validation
    """
    group_size = len(A)//10+1

    train_losses = []
    test_losses = []

    train_correlations = []
    test_correlations = []
    for i in range(0, len(A), group_size):
        print('Run {}/10...'.format(i//group_size + 1))

        A_test = A[i: i+group_size]
        A_train = A[:i] + A[i+group_size:]

        train_acc, test_acc, train_corr, test_corr = train_model(A_train,
                                                                 A_test,
                                                                 params,
                                                                 verbose)
        train_losses.append(train_acc)
        test_losses.append(test_acc)
        train_correlations.append(train_corr)
        test_correlations.append(test_corr)

    print('Train accuracy:', np.mean(train_losses), '+- ', np.std(train_losses))
    print('Test accuracy:', np.mean(test_losses), '+- ', np.std(test_losses))
    print('Train correlation:', np.mean(train_correlations), '+- ', np.std(train_correlations))
    print('Test correlation:', np.mean(test_correlations), '+- ', np.std(test_correlations))

# Experiments (Inner)

In [ ]:
torch.manual_seed(42)

## Graph Edit Distance

In [ ]:
A = get_synth_data(max_size=9)

### GNN

In [ ]:
params = {
    'input_dim': 10,
    'hidden_dim': 64,
    'output_dim': 10,
    'n_layers': 4,
    'epochs': 15,
    'model': GNN,
    'mode': 'inner',
    'lr': 0.001,
    'weight_decay': 0,
    'max_patience': 5,
    'dropout': 0.3,
    'batch_size': 5,
    'similarity_measure': 'ged'
}

cross_val(A, params, verbose=False)

### GIN


In [ ]:
params = {
    'input_dim': 10,
    'hidden_dim': 64,
    'output_dim': 10,
    'n_layers': 4,
    'epochs': 15,
    'model': GIN,
    'mode': 'inner',
    'lr': 0.001,
    'weight_decay': 0,
    'max_patience': 5,
    'dropout': 0.3,
    'batch_size': 5,
    'similarity_measure': 'ged'
}

cross_val(A, params, verbose=False)

### GNN (GR)


In [ ]:
params = {
    'input_dim': 10,
    'hidden_dim': 64,
    'output_dim': 10,
    'n_layers': 4,
    'epochs': 15,
    'model': GNNGlobalReadout,
    'mode': 'inner',
    'lr': 0.001,
    'weight_decay': 0,
    'max_patience': 5,
    'dropout': 0.3,
    'batch_size': 5,
    'similarity_measure': 'ged'
}

cross_val(A, params, verbose=False)

### Baselines


In [ ]:
params = {
    'input_dim': 10,
    'hidden_dim': 64,
    'output_dim': 10,
    'n_layers': 4,
    'epochs': 15,
    'model': RandomBaseline,
    'mode': 'inner',
    'lr': 0.001,
    'weight_decay': 0,
    'max_patience': 5,
    'dropout': 0.3,
    'batch_size': 5,
    'similarity_measure': 'ged'
}

cross_val(A, params, verbose=False)

In [ ]:
params = {
    'input_dim': 10,
    'hidden_dim': 64,
    'output_dim': 10,
    'n_layers': 4,
    'epochs': 15,
    'model': ConstantBaseline,
    'mode': 'inner',
    'lr': 0.001,
    'weight_decay': 0,
    'max_patience': 5,
    'dropout': 0.3,
    'batch_size': 5,
    'similarity_measure': 'ged'
}

cross_val(A, params, verbose=False)

## Feature Distance

In [ ]:
A = get_synth_data(max_size=9)

### GNN

In [ ]:
params = {
    'input_dim': 10,
    'hidden_dim': 64,
    'output_dim': 10,
    'n_layers': 4,
    'epochs': 5,
    'model': GNN,
    'mode': 'inner',
    'lr': 0.001,
    'weight_decay': 0,
    'max_patience': 5,
    'dropout': 0.3,
    'batch_size': 5,
    'similarity_measure': 'feats'
}

cross_val(A, params, verbose=False)

### GIN

In [ ]:
params = {
    'input_dim': 10,
    'hidden_dim': 64,
    'output_dim': 10,
    'n_layers': 4,
    'epochs': 5,
    'model': GIN,
    'mode': 'inner',
    'lr': 0.001,
    'weight_decay': 0,
    'max_patience': 5,
    'dropout': 0.3,
    'batch_size': 5,
    'similarity_measure': 'feats'
}

cross_val(A, params, verbose=False)

### GNN (GR)


In [ ]:
params = {
    'input_dim': 10,
    'hidden_dim': 64,
    'output_dim': 10,
    'n_layers': 4,
    'epochs': 5,
    'model': GNNGlobalReadout,
    'mode': 'inner',
    'lr': 0.001,
    'weight_decay': 0,
    'max_patience': 5,
    'dropout': 0.3,
    'batch_size': 5,
    'similarity_measure': 'feats'
}

cross_val(A, params, verbose=False)

### Baselines


In [ ]:
params = {
    'input_dim': 10,
    'hidden_dim': 64,
    'output_dim': 10,
    'n_layers': 4,
    'epochs': 15,
    'model': RandomBaseline,
    'mode': 'inner',
    'lr': 0.001,
    'weight_decay': 0,
    'max_patience': 5,
    'dropout': 0.3,
    'batch_size': 5,
    'similarity_measure': 'feats'
}

cross_val(A, params, verbose=False)

In [ ]:
params = {
    'input_dim': 10,
    'hidden_dim': 64,
    'output_dim': 10,
    'n_layers': 4,
    'epochs': 15,
    'model': ConstantBaseline,
    'mode': 'inner',
    'lr': 0.001,
    'weight_decay': 0,
    'max_patience': 5,
    'dropout': 0.3,
    'batch_size': 5,
    'similarity_measure': 'feats'
}

cross_val(A, params, verbose=False)